In [1]:
import numpy as np
from pathlib import Path
import bw2data as bd
import bw2calc as bc
from gsa_framework.utils import read_pickle, write_pickle
from gsa_framework.models.life_cycle_assessment import LCAModel
from dev.utils_graph_traversal import filter_uncertain_technosphere_exchanges


In [7]:
if __name__ == "__main__":

#     path_base = Path(
#         "/Users/akim/PycharmProjects/gsa-framework-master/dev/write_files/"
#     )
    path_base = Path('/data/user/kim_a/protocol_gsa')
    write_dir = path_base 
    write_dir_sct = write_dir / "supply_chain_paper3"
    write_dir_sct.mkdir(exist_ok=True,parents=True)

    bd.projects.set_current("GSA for protocol")
    co = bd.Database("CH consumption 1.0")
    demand_act = [act for act in co if "Food" in act["name"]]
    assert len(demand_act) == 1
    demand_act = demand_act[0]
    demand = {demand_act: 1}
    uncertain_method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    lca = bc.LCA(demand, uncertain_method)
    lca.lci()
    lca.lcia()
    print(lca.score)

    cutoff = 1e-7
    max_calc = 1e8
    sct_tech_params_filename = "tech_params_cutoff{}_maxcalc{}.pickle".format(cutoff, int(max_calc))
    sct_tech_params_filepath = write_dir_sct / sct_tech_params_filename
    if not sct_tech_params_filepath.exists():
        tech_params_sct = filter_uncertain_technosphere_exchanges(lca, cutoff=cutoff, max_calc=max_calc)
        write_pickle(tech_params_sct, sct_tech_params_filepath)
    else:
        tech_params_sct = read_pickle(sct_tech_params_filepath)

211.57670115973556
TECHNOSPHERE (19353, 19353) filtering resulted in 97900 of 236414 UNCERTAIN exchanges and took 120311 iterations in 1057.82 seconds.
